In [2]:
import math
import numpy as np
import pandas as pd
import pdb

import torch
import torch.nn.functional as F
import torch.distributions
import torch.optim as optim
from torch import nn 
from torch.autograd import Variable

import torchtext
from torchtext import data
from torchtext import datasets
from torchtext.vocab import Vectors, GloVe

In [3]:
class Model(torch.nn.Module):
    def __init__(self, batch_size, num_classes, mlp_out_size, vocab_size, embedding_length, weights, hidden_size = 100, biDirectional = False):
        super(Model, self).__init__() 
        """
        Arguments
        ---------
        batch_size : Size of the batch which is same as the batch_size of the data returned by the TorchText BucketIterator
        output_size : 28 = (For full classification)
        hidden_sie : Size of the hidden_state of the LSTM   (// Later BiLSTM)
        vocab_size : Size of the vocabulary containing unique words
        embedding_length : Embeddding dimension of GloVe word embeddings
        weights : Pre-trained GloVe word_embeddings which we will use to create our word_embedding look-up table 
        --------

        """

        self.batch_size = batch_size
        self.num_classes = num_classes
        self.vocab_size = vocab_size
        self.embedding_length = embedding_length
        self.hidden_size = hidden_size

        self.mlp_out_size = mlp_out_size
        self.biDirectional = biDirectional

        self.word_embeddings = nn.Embedding(vocab_size, embedding_length)
        self.word_embeddings.weights = nn.Parameter(weights, requires_grad=False)
        
        self.lstm_layer = LSTM(self.batch_size, self.hidden_size, self.embedding_length, self.biDirectional)

        if(self.biDirectional):
#             self.mlp = MLP(self.hidden_size*2, self.mlp_out_size)
            self.FF = nn.Linear(self.hidden_size*2, num_classes)
        else:
#             self.mlp = MLP(self.hidden_size, self.mlp_out_size)
            self.FF = nn.Linear(self.hidden_size, self.num_classes)

    def forward(self, input_sequence):
        input_ = self.word_embeddings(input_sequence)
        out_lstm, final_hidden_state = self.lstm_layer(input_)
        if self.biDirectional:
            final_hidden_state = final_hidden_state.view(2, 2, input_.shape[0], self.hidden_size) # num_layer x num_dir x batch x hidden
            final_hidden_state = final_hidden_state[-1]
            final_hidden_state = final_hidden_state.transpose(0,1).reshape(input_.shape[0], self.hidden_size*2)
        else:
            final_hidden_state = final_hidden_state[-1]
        
        ff_output = self.FF(final_hidden_state)
#         print("FF out size: ", ff_output.shape)
        predictions = torch.softmax(ff_output, dim = -1)
        return predictions

In [87]:
class LSTM(torch.nn.Module):
    """
        Arguments
        ---------
        batch_size : Size of the batch which is same as the batch_size of the data returned by the TorchText BucketIterator
        hidden_size : Size of the hidden_state of the LSTM   (* Later BiLSTM, check dims for BiLSTM *)
        embedding_length : Embeddding dimension of GloVe word embeddings
        --------
    """
    def __init__(self, batch_size, hidden_size, embedding_length, biDirectional = False, num_layers = 2):

        super(LSTM, self).__init__()
        self.batch_size = batch_size
        self.hidden_size = hidden_size
        self.embedding_length = embedding_length
        self.biDirectional	= biDirectional
        self.num_layers = num_layers

        self.lstm = nn.LSTM(self.embedding_length, self.hidden_size, bidirectional = self.biDirectional, batch_first = True, num_layers = self.num_layers)   # Dropout  

    def forward(self, input_sequence, batch_size=None):
        out_lstm, (final_hidden_state, final_cell_state) = self.lstm(input_sequence)   # ouput dim: ( batch_size x seq_len x hidden_size )
        return out_lstm, final_hidden_state

In [88]:
# If want to add extra MLP Layer
class MLP(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MLP, self).__init__()

        self.input_dim = input_dim
        self.output_dim = output_dim

        self.ff_1 = nn.Linear(self.input_dim, self.output_dim)
        self.relu = nn.ReLU()
        self.ff_2 = nn.Linear(self.output_dim,1)
        self.sigmoid = nn.Sigmoid()

    def forward(self,x):
        out_1 = self.ff_1(x)
        out_relu = self.relu(out_1)
        out_2 = self.ff_2(out_relu)
        out_sigmoid = self.sigmoid(out_2)

        return out_sigmoid

In [89]:
def clip_gradient(model, clip_value):
    params = list(filter(lambda p: p.grad is not None, model.parameters()))
    for p in params:
        p.grad.data.clamp_(-clip_value, clip_value)

In [103]:
def train_model(model, optim, train_iter, epoch, batch_size, num_classes):
    total_epoch_loss = 0
    total_epoch_acc = 0
    
    steps = 0
    model.train()
    for idx, batch in enumerate(train_iter):
        text = batch.text[0]
        target = batch.labels
        target = torch.autograd.Variable(target).long()
        if torch.cuda.is_available():
            text = text.cuda()
            target = target.cuda()
        if (text.size()[0] is not batch_size): # One of the batch returned by BucketIterator has length different than 32.
            continue
        optim.zero_grad()
        prediction = model(text)
#         print(prediction.shape)
#         print(prediction)
#         print(target.shape)
#         print(target)
        loss =  loss_fn(prediction, target)
        if math.isnan(loss.item()):
            print(prediction, target)

        num_corrects = (torch.max(prediction, 1)[1].view(target.size()).data == target.data).sum()
        acc = 100.0 * num_corrects/len(batch)
        total_epoch_loss += loss.item()
        total_epoch_acc += acc.item()
        
    return total_epoch_loss/len(train_iter), total_epoch_acc/len(train_iter)

In [104]:
def eval_model(model, val_iter, num_classes):
    total_epoch_loss = 0
    total_epoch_acc = 0
    total_attention =  0
    total_samples = 0 
    model.eval()
    with torch.no_grad():
        for idx, batch in enumerate(val_iter):
            text = batch.text[0]
            if (text.size()[0] is not 32):
                continue
            target = batch.labels
            target = torch.autograd.Variable(target).long()
            if torch.cuda.is_available():
                text = text.cuda()
                target = target.cuda()
            output = model(text)
            print("Output", output)
            prediction = model(text)
            # Sanity check
            # print("Test Prediction: ", prediction)

            # Defualt - Cross entropy loss funtion
            loss =  loss_fn(prediction, target)
            
            if math.isnan(loss.item()):
                print(prediction, target)
            
            num_corrects = (torch.max(prediction, 1)[1].view(target.size()).data == target.data).sum()
            acc = 100.0 * num_corrects/len(batch)
            total_epoch_loss += loss.item()
            total_epoch_acc += acc.item()
            
    return total_epoch_loss/len(val_iter), total_epoch_acc/len(val_iter)

<>:10: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:10: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<ipython-input-104-800342ba515b>:10: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if (text.size()[0] is not 32):


In [101]:
def load_data(batch_size= 16, embedding_length = 100):
    tokenize = lambda x: x.split()
    TEXT = data.Field(sequential=True, tokenize=tokenize, lower=True, include_lengths=True, batch_first=True, fix_length=30)
    LABELS = data.LabelField(batch_first=True, dtype=torch.float)

    train, val, test = data.TabularDataset.splits(
      path='/Users/prakruti/Documents/GoEmotions-classification/data/labelled_data', train='train.tsv',
      validation='dev.tsv', test='test.tsv', format='tsv',
      fields=[('text', TEXT), ('labels', LABELS)])
    
    train_iter, val_iter, test_iter = data.BucketIterator.splits(
      (train, val, test), batch_sizes=(16, 16, 16), device=0)

    # build the vocabulary
    TEXT.build_vocab(train, vectors=GloVe(name='6B', dim=embedding_length))
    LABELS.build_vocab(train)
    print(LABELS.vocab.__dict__)

    word_embeddings = TEXT.vocab.vectors
    vocab_size = len(TEXT.vocab)

    return TEXT, vocab_size, word_embeddings, train_iter, val_iter, test_iter

In [93]:
# def vectorize(x):
#     if x == '':
#         return 5
#     else:
#         x = float(x)
#         return int(x)

In [94]:
TEXT, vocab_size, word_embeddings, train_iter, valid_iter, test_iter = load_data()

The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


{'freqs': Counter({'2': 16327, '5': 14219, '0': 5829, '4': 3888, '3': 2489, '1': 658}), 'itos': ['2', '5', '0', '4', '3', '1'], 'unk_index': None, 'stoi': defaultdict(None, {'2': 0, '5': 1, '0': 2, '4': 3, '3': 4, '1': 5}), 'vectors': None}


In [95]:
next(iter(train_iter))


[torchtext.data.batch.Batch of size 16]
	[.text]:('[torch.LongTensor of size 16x30]', '[torch.LongTensor of size 16]')
	[.labels]:[torch.FloatTensor of size 16]

In [96]:
# Over-writing the loss function to simple cross entropy loss
loss_fn = F.cross_entropy

learning_rate = 2e-4
batch_size = 16
output_size = 2
embedding_length = 100
num_classes = 6
mlp_out_size = 32
weights = word_embeddings
hidden_size = 100

model = Model(batch_size, num_classes, mlp_out_size, vocab_size, embedding_length, weights, hidden_size, biDirectional=False)
optim = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()))

In [105]:
num_bad_epochs = 0
epoch = 0
least_loss = float('inf')
training_stats = pd.DataFrame(columns=['Epoch', 'Train_Loss', 'Train_Acc', 'Val_Loss', 'Val_Acc'])

while(True):
    print("Epoch", epoch)
    train_loss, train_acc = train_model(model, optim, train_iter, epoch, batch_size, num_classes)
    print("Train Accuracy", train_acc)
    print("Train loss", train_loss)
    val_loss, val_acc = eval_model(model, valid_iter, num_classes) 
    print(f'Epoch: {epoch+1:02}')
    if val_loss < least_loss:
        least_loss = val_loss
        num_bad_epochs = 0
        print("*** Least validation loss")
        torch.save(model.state_dict(), "LSTM")
    else:
        num_bad_epochs += 1
#     print(f'Epoch: {epoch+1:2}, Train Loss: {train_loss:.3f}, Train Acc: {train_acc:.2f}%')
    print(f'Train Loss: {train_loss:.3f}, Train Acc: {train_acc:.2f}%') 
    print(f'Val Loss: {val_loss:3f}, Val Acc: {val_acc:.2f}%')
    print("-------------")
    
    training_stats = training_stats.append(
        pd.Series([epoch+1, train_loss, train_acc, val_loss, val_acc], index=training_stats.columns), 
        ignore_index=True)
    if num_bad_epochs >= 10:
        break
        
    epoch += 1
    if epoch == 10:
        break

Epoch 0
Train Accuracy 1.529108327192336
Train loss 1.792445732067856


TypeError: '<' not supported between instances of 'Example' and 'Example'